In [14]:
from gurobipy import *
from math import *
from math import floor
import numpy as np
import pandas as pd
import timeit
import xlwt as xlwt
from xlwt import Workbook

In [2]:
num_ac_types = 3 # Number of aircraft types
LF = 0.8 # Load factor
BT = 10 * 7 # Block time [h/week] - average utilisation time (same for all aircraft types k)
f = 1.42 # Fuel cost [USD/gallon]

In [3]:
data_demand = pd.read_csv('Group_6_Demand.csv')
q = np.array(data_demand)[::,1::]  # Demand (matrix) between airports i and j
demand = q

In [4]:
airports_data = pd.read_csv('Group_6_Airport_info.csv')
Airports = airports_data['ICAO Code'].tolist()
airports = range(len(Airports))
runway_lengths = airports_data['Runway (m)'].tolist()

In [5]:
# Creating matrix with max runway length between origin and destination
runways = np.zeros((len(runway_lengths), len(runway_lengths)))
for origin in range(len(runway_lengths)):
    for destination in range(len(runway_lengths)):
        if origin != destination:
            if runway_lengths[origin] < runway_lengths[destination]:
                runways[origin][destination] = runway_lengths[origin]
            else:
                runways[origin][destination] = runway_lengths[destination]

In [6]:
# Extracting the distance between airports i and j from Excel file 'results.xlsx'

data_distances = pd.read_csv('Group_6_Distances.csv')
distances = np.array(data_distances)[::,1::]  # Distances (matrix) between airports i and j

In [7]:
# Aircraft parameters
ac_param = pd.read_csv('Aircraft_info.csv')
speed = ac_param['Speed'].tolist()
seats = ac_param['Seats'].tolist()
TAT = ac_param['TAT'].tolist()
ac_range = ac_param['Range'].tolist()
runway = ac_param['Runway'].tolist()
lease = ac_param['Lease_c'].tolist()
op_cost = ac_param['Operating_c'].tolist()
time_cost_param = ac_param['Time_c'].tolist()
fuel_cost_param = ac_param['Fuel_c'].tolist()

keys = ['speed', 'seats', 'TAT', 'ac_range', 'runway', 'lease', 'op_cost', 'time_cost_param', 'fuel_cost_param']
values = [speed, seats, TAT, ac_range, runway, lease, op_cost, time_cost_param, fuel_cost_param]
Aircraft = dict(zip(keys, values))
print(Aircraft)

{'speed': [820, 850, 870], 'seats': [70, 150, 320], 'TAT': [35, 45, 60], 'ac_range': [3300, 6300, 12000], 'runway': [1600, 1800, 2600], 'lease': [34000, 70000, 180000], 'op_cost': [600, 1250, 2000], 'time_cost_param': [775, 1400, 2800], 'fuel_cost_param': [2.0, 3.75, 7.0]}


In [8]:
# Create a new list for 'TAT' converted to hours
converted_TAT = [tat / 60 for tat in Aircraft['TAT']]

# Update the 'TAT' value in the Aircraft dictionary with the converted values
Aircraft['TAT'] = converted_TAT


# Yield between airports i and j - Revenue per RPK flown
Yield = np.zeros((len(Airports),len(Airports)))
for i in range(len(distances)):
    for j in range(len(distances)):
        if i != j:
            Yield[i][j] = 5.9 * distances[i][j] ** (-0.76) + 0.043

In [9]:
# Range matrix
a = {}
for k in range(num_ac_types):
    for i in range(len(distances)):
        for j in range(len(distances[i])):
            a[i,j,k] = 0
            if distances[i][j] <= Aircraft['ac_range'][k] :
                a[i,j,k] = 10000


In [10]:
# Runway matrix
run = {}
for k in range(num_ac_types):
    for i in range(len(runways)):
        for j in range(len(runways[i])):
            run[i,j,k] = 0
            if runways[i][j] >= Aircraft['runway'][k] :
                run[i,j,k] = 10000

## Optimziation model.

In [15]:
m = Model('model')

# Decision variables
x = {} # direct pax flow from airport i to airport j
w = {} # pax flow from airport i to airport j with transfer at hub
z = {} # number of flights (frequency) from airport i to airport j
AC = {} # number of aircraft of type k

# Setting hub in Göteborg
g = np.ones(34) # 34 airports
g[0] = 0 # Stockholm (ESGG) is the hub

# Creating dictionaries of hub & spokes
hub = {}
spokes = {}
for j in range(len(q)):
    if g[j] == 0: # Hub is the first one in the list
        hub[j] = j
    else:
        spokes[j] = j

for i in airports:
    for j in airports:
        x[i,j] = m.addVar(obj = Yield[i][j] * distances[i][j], lb=0, vtype=GRB.INTEGER)
        w[i,j] = m.addVar(obj = 0.9 * Yield[i][j] * distances[i][j], lb=0, vtype=GRB.INTEGER) # 10% lower revenue in case of transfer at hub

        for k in range(num_ac_types):
            C_O_ac = Aircraft['op_cost'][k]
            C_T_ac = Aircraft['time_cost_param'][k] * distances[i][j] / Aircraft['speed'][k]
            C_F_ac = Aircraft['fuel_cost_param'][k] * f * distances[i][j] / 1.5

            if g[i] == 0 or g[j] == 0: # if origin/destination is the hub
                z[i,j,k] = m.addVar(obj= -0.7 * (C_O_ac + C_T_ac + C_F_ac), lb=0, vtype=GRB.INTEGER) # 30% lower operating costs when hub is origin/destination
            else:
                z[i,j,k] = m.addVar(obj= - (C_O_ac + C_T_ac + C_F_ac), lb=0, vtype=GRB.INTEGER)

for k in range(num_ac_types):
    AC[k] =  m.addVar(obj= - (Aircraft['lease'][k]), lb=0, vtype=GRB.INTEGER)

m.update()
m.setObjective(m.getObjective(), GRB.MAXIMIZE)



for i in airports:
    for j in airports:
        # C1: Demand constraint
        m.addConstr(x[i,j] + w[i,j] <= q[i][j])

        # C1*: Transfer pax constraint (also demand-related)
        m.addConstr(w[i,j] <= q[i][j] * g[i] * g[j])

        # C2: Capacity constraint
        m.addConstr(x[i,j] + quicksum(w[i,m] for m in range(len(demand))) * (1 - g[j]) + quicksum(w[m,j] for m in range(len(demand))) * (1 - g[i])
                    <= quicksum(z[i,j,k] * floor(Aircraft['seats'][k] * LF) for k in range(num_ac_types)))

        # C5: Range constraint
        for k in range(num_ac_types):
            m.addConstr(z[i,j,k] <= a[i,j,k])

        # "C6": Runway length constraint (new!)
        for k in range(num_ac_types):
            m.addConstr(z[i,j,k] <= run[i,j,k])

    # C3: In-out constraint
    for k in range(num_ac_types):
        m.addConstr(quicksum(z[i,j,k] for j in range(len(demand[i]))) == quicksum(z[j,i,k] for j in range(len(demand[i]))))

# C4: Time constraint
for k in range(num_ac_types):
    m.addConstr(quicksum(quicksum((distances[i][j] / Aircraft['speed'][k] + 1.5 * Aircraft['TAT'][k]) * z[i,j,k] for j in hub) for i in range(len(demand)))
                + quicksum(quicksum((distances[i][j] / Aircraft['speed'][k] + Aircraft['TAT'][k]) * z[i,j,k] for j in spokes) for i in range(len(demand)))
                <= BT * AC[k])

m.update()

m.optimize()
m.write("testout.sol")
status = m.status


if status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')

elif status == GRB.Status.OPTIMAL or True:
    f_objective = m.objVal
    print('***** RESULTS ******')
    print('\nObjective Function Value: \t %g' % f_objective)

elif status != GRB.Status.INF_OR_UNBD and status != GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2447081 - for non-commercial use only - registered to yf___@tudelft.net
Optimize a model with 9903 rows, 5448 columns and 25940 nonzeros
Model fingerprint: 0xf6798c41
Variable types: 0 continuous, 5448 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-01, 3e+02]
  Objective range  [2e+01, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 8062 rows and 1832 columns
Presolve time: 0.02s
Presolved: 1841 rows, 3616 columns, 11494 nonzeros
Variable types: 0 continuous, 3616 integer (1 binary)

Root relaxation: objective 3.089223e+05, 2159 iterations, 0.06 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  

## Printing solutions.

In [16]:

print()
print("Frequencies:----------------------------------")
print()
for i in airports:
    for j in airports:
        for k in range(num_ac_types):
            if z[i,j,k].X >0:
                print(Airports[i], ' to ', Airports[j], z[i,j,k].X)


Frequencies:----------------------------------

LIMC  to  EDDF 1.0
LIMC  to  EDDF 4.0
LIMC  to  RJAA 1.0
LIMC  to  KJFK 1.0
LIMC  to  SBGL 3.0
LIMC  to  LEMD 3.0
LIMC  to  LTFM 1.0
LIMC  to  SAEZ 5.0
LIMC  to  ZSPD 2.0
LIMC  to  EDDM 12.0
LIMC  to  EDDM 2.0
LIMC  to  LFPG 6.0
LIMC  to  VIDP 2.0
LIMC  to  OMDB 2.0
LIMC  to  EDDT 1.0
LIMC  to  EDDT 2.0
LIMC  to  MMMX 3.0
LIMC  to  EFHK 1.0
LIMC  to  EBBR 1.0
LIMC  to  EGPH 3.0
LIMC  to  LKPR 2.0
LIMC  to  FACT 5.0
LIMC  to  LGAV 4.0
LIMC  to  BIKF 3.0
LIMC  to  ESSA 2.0
LIMC  to  EIDW 1.0
LIMC  to  LPPT 4.0
LIMC  to  EETN 2.0
LIMC  to  LFQQ 1.0
LIMC  to  EFRO 1.0
LIMC  to  LDZA 3.0
LIMC  to  EHEH 2.0
EDDF  to  LIMC 4.0
EDDF  to  EDDM 1.0
EDDF  to  EDDM 1.0
EDDF  to  LFPG 2.0
EDDF  to  EDDT 1.0
EDDF  to  EGPH 4.0
EDDF  to  LKPR 1.0
EDDF  to  LFQQ 1.0
EDDF  to  LDZA 1.0
EDDF  to  EHEH 1.0
RJAA  to  LIMC 1.0
KJFK  to  LIMC 1.0
SBGL  to  LIMC 3.0
LEMD  to  LIMC 3.0
LEMD  to  LPPT 1.0
LTFM  to  LIMC 1.0
SAEZ  to  LIMC 5.0
ZSPD  to  LIMC 2.0


## Excel with the output.

In [17]:
wb = Workbook()
overview = wb.add_sheet('Overview', cell_overwrite_ok = True)
sheet_x = wb.add_sheet('x', cell_overwrite_ok = True)
sheet_w = wb.add_sheet('w', cell_overwrite_ok = True)
for k in range(num_ac_types):
    locals()['sheet_{0}'.format(k)] = wb.add_sheet('AC %s' %(k+1), cell_overwrite_ok = True)

# Writing overview
overview.write(0, 0, "Overview")
overview.write(1, 0, "profit [EUR/week]")
overview.write(2, 0, "gap [%]")
overview.write(1, 1, m.objVal)
overview.write(2, 1, m.MIPgap * 100)

for k in range(num_ac_types):
    overview.write(3 + k, 0, 'Aircraft %s' % str(k+1))
    overview.write(3 + k, 1, AC[k].x)

# Writing results in the Excel file 'Model_1B_results.xls'
sheet_x.write(0, 0, 'x[i,j]')
sheet_w.write(0, 0, 'w[i,j]')

for k in range(num_ac_types):
    locals()['sheet_{0}'.format(k)].write(0, 0, 'aircraft %s' %(k+1))

for i in range(len(demand)):
    sheet_x.write(i + 1, 0, Airports[i])
    sheet_w.write(i + 1, 0, Airports[i])

    for k in range(num_ac_types):
        locals()['sheet_{0}'.format(k)].write(i+1, 0, Airports[i])

    for j in range(len(demand[i])):
        sheet_x.write(0, j + 1, Airports[j])
        sheet_w.write(0, j + 1, Airports[j])

        sheet_x.write(i + 1, j + 1, x[i, j].x)
        sheet_w.write(i + 1, j + 1, w[i, j].x)

        for k in range(num_ac_types):
            locals()['sheet_{0}'.format(k)].write(0, j+1, Airports[j])
            locals()['sheet_{0}'.format(k)].write(i+1, j+1, z[i,j,k].x)

# Saving the Excel file
wb.save('Assignment1_Problem1_results.xls')